# Central-spin worked example: derivations and cross-checks

This notebook reproduces key quantities used in the observer-quality paper:
- conditional fragment overlaps in a pure-dephasing central-spin model,
- Chernoff-style discrimination exponent in the ideal record limit,
- sample-size requirements as a function of calibration and ε-SBS slack.

It is intended as a transparent numerical cross-check of the analytic expressions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def fragment_overlap(g, t):
    """Idealized central-spin pure-dephasing overlap for a single environment spin.
    For coupling g and time t, one common toy model yields overlap |cos(2 g t)|.
    """
    return np.abs(np.cos(2.0*g*t))

def chernoff_exponent_from_overlap(c):
    """For two pure states with overlap c, the (quantum) Chernoff exponent is -log(min_s Tr rho^s sigma^{1-s}).
    For pure states, Tr rho^s sigma^{1-s} = Tr rho sigma = |<psi|phi>|^2 = c^2.
    Hence exponent = -log(c^2) = -2 log c.
    """
    c = np.clip(c, 1e-12, 1.0)
    return -2.0*np.log(c)


In [ ]:
# Deterministic couplings
rng = np.random.default_rng(0)
N = 50
g = rng.uniform(0.05, 0.25, size=N)
t = np.linspace(0.0, 40.0, 400)

# Compute typical overlap per fragment (median)
c_med = np.median(fragment_overlap(g[:,None], t[None,:]), axis=0)
xi = chernoff_exponent_from_overlap(c_med)
xi[:5], xi.max()


In [ ]:
plt.figure()
plt.plot(t, xi)
plt.xlabel('time t')
plt.ylabel('Chernoff exponent (toy)')
plt.title('Toy Chernoff exponent vs time (median coupling)')
plt.show()


In [ ]:
def m_required(delta, eps, xi):
    """Sufficient sample size for error <= delta given ε-SBS slack eps and exponent xi.
    Uses exp(-m*xi) + eps <= delta -> m >= log(1/(delta-eps))/xi.
    """
    if delta <= eps:
        return np.inf
    return np.ceil(np.log(1.0/(delta-eps))/np.maximum(xi, 1e-12))

delta = 1e-3
eps = 1e-4
m = m_required(delta, eps, xi)
np.nanmin(m[np.isfinite(m)]), np.nanmax(m[np.isfinite(m)])


In [ ]:
plt.figure()
plt.plot(t, m)
plt.yscale('log')
plt.xlabel('time t')
plt.ylabel('m_required (log scale)')
plt.title('Sample size requirement vs time (toy)')
plt.show()
